## 預測價格

In [8]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed

# --------------------------
# 1. 讀取資料
# --------------------------
file_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv'
data = pd.read_csv(file_path)

eco_raw = data[data['艙等（主航段）'] == '經濟艙'].copy()
biz_raw = data[data['艙等（主航段）'] == '商務艙'].copy()

# --------------------------
# 2. 欄位定義
# --------------------------
cat_cols = [
    '出發時段','出發機場代號','抵達時段','航空聯盟組合','航空公司組合',
    '航空聯盟','停靠站數量','機型分類','假期','飛行時間兩段分類','是否為平日'
]
num_cols = ['停留時間_分鐘','實際飛行時間_分鐘','經濟指標','機場指標','competing_flights']
log_col  = '平均價格_log'
true_col = '平均價格'

def encode_and_scale(df_raw):
    # 只保留需要的欄位
    cols = cat_cols + num_cols + [log_col, true_col]
    df_sub = df_raw[cols]
    # Dummy encoding
    df_enc = pd.get_dummies(df_sub, columns=cat_cols, drop_first=True)
    # 標準化
    scaler = StandardScaler()
    df_enc[num_cols] = scaler.fit_transform(df_enc[num_cols])
    return df_enc, scaler

# 編碼 & 標準化
eco_enc, eco_scaler = encode_and_scale(eco_raw)
biz_enc, biz_scaler = encode_and_scale(biz_raw)

# --------------------------
# 3. 對齊特徵欄位
# --------------------------
# 取經濟艙與商務艙共同的特徵欄位
common_feats = list(set(eco_enc.columns) & set(biz_enc.columns))
# 移除 log 與真實價格欄
for c in [log_col, true_col]:
    if c in common_feats:
        common_feats.remove(c)

# --------------------------
# 4. 模型訓練 & 評估
# --------------------------
seed = 123
# 經濟艙模型
eco_model = XGBRegressor(n_estimators=100, random_state=seed, n_jobs=-1)
eco_model.fit(eco_enc[common_feats], eco_enc[log_col])
eco_log_pred = eco_model.predict(eco_enc[common_feats])
print(f"[經濟艙] log-space MSE={mean_squared_error(eco_enc[log_col], eco_log_pred):.4f}, "
      f"R2={r2_score(eco_enc[log_col], eco_log_pred):.4f}")
sigma2_eco = np.mean((eco_enc[log_col] - eco_log_pred)**2)

# 商務艙模型
biz_model = XGBRegressor(n_estimators=100, random_state=seed, n_jobs=-1)
biz_model.fit(biz_enc[common_feats], biz_enc[log_col])
biz_log_pred = biz_model.predict(biz_enc[common_feats])
print(f"[商務艙] log-space MSE={mean_squared_error(biz_enc[log_col], biz_log_pred):.4f}, "
      f"R2={r2_score(biz_enc[log_col], biz_log_pred):.4f}")
sigma2_biz = np.mean((biz_enc[log_col] - biz_log_pred)**2)

# --------------------------
# 5. 擷取真實組合
# --------------------------
def get_real_combinations(df_raw, code, cabin):
    sub = df_raw[(df_raw['抵達機場代號'] == code) & (df_raw['艙等（主航段）'] == cabin)]
    cols = cat_cols + num_cols + [true_col]
    return sub[cols].drop_duplicates().reset_index(drop=True)

# --------------------------
# 6. 預測並存檔
# --------------------------
def predict_and_save(df_raw, model, scaler, sigma2, cabin, prefix, airports):
    out_dir = 'predict_data/long'
    os.makedirs(out_dir, exist_ok=True)
    for code in airports:
        combos = get_real_combinations(df_raw, code, cabin)
        # 編碼 & 補齊欄位
        sub = combos[cat_cols + num_cols]
        enc = pd.get_dummies(sub, columns=cat_cols, drop_first=True)
        enc = enc.reindex(columns=common_feats, fill_value=0)
        # 標準化
        enc[num_cols] = scaler.transform(combos[num_cols])
        # 並行預測
        log_preds = np.concatenate(
            Parallel(n_jobs=-1)(
                delayed(model.predict)(enc.iloc[i:i+500])
                for i in range(0, len(enc), 500)
            )
        )
        # bias-corrected back-transform
        combos['預測_平均價格'] = np.expm1(log_preds + 0.5 * sigma2)
        combos['抵達機場代號'] = code
        # 去重
        combos = combos.drop_duplicates(subset=cat_cols + num_cols)
        # 存檔
        fn = os.path.join(out_dir, f"{prefix}_{code}.csv")
        combos.to_csv(fn, index=False)
        print(f"✅ 已存：{fn} （共 {len(combos)} 筆）")

# --------------------------
# 7. 執行預測
# --------------------------
airports = sorted(data['抵達機場代號'].unique())
predict_and_save(data, eco_model, eco_scaler, sigma2_eco, '經濟艙', 'eco', airports)
predict_and_save(data, biz_model, biz_scaler, sigma2_biz, '商務艙', 'biz', airports)

[經濟艙] log-space MSE=0.0254, R2=0.9171
[商務艙] log-space MSE=0.0124, R2=0.9063
✅ 已存：predict_data/long/eco_CDG.csv （共 3952 筆）
✅ 已存：predict_data/long/eco_FRA.csv （共 5733 筆）
✅ 已存：predict_data/long/eco_JFK.csv （共 3491 筆）
✅ 已存：predict_data/long/eco_LAX.csv （共 6189 筆）
✅ 已存：predict_data/long/eco_LHR.csv （共 7092 筆）
✅ 已存：predict_data/long/eco_SYD.csv （共 5104 筆）
✅ 已存：predict_data/long/eco_ZRH.csv （共 3501 筆）
✅ 已存：predict_data/long/biz_CDG.csv （共 3207 筆）
✅ 已存：predict_data/long/biz_FRA.csv （共 4625 筆）
✅ 已存：predict_data/long/biz_JFK.csv （共 3446 筆）
✅ 已存：predict_data/long/biz_LAX.csv （共 5991 筆）
✅ 已存：predict_data/long/biz_LHR.csv （共 5490 筆）
✅ 已存：predict_data/long/biz_SYD.csv （共 4387 筆）
✅ 已存：predict_data/long/biz_ZRH.csv （共 2774 筆）
